In [2]:
import pandas as pd
import numpy as np
from scipy import spatial
import math
import polars as pl
import csv
import random
import tqdm

In [3]:
weapons = ["bomb", "rifle", "arrow"]

In [5]:
arr_random_input = np.random.default_rng().uniform(low=-100,high=100,size=[1000000,2])
arr_random_query = np.random.default_rng().uniform(low=-100,high=100,size=[1000,2])
df = pd.DataFrame(arr_random_input, columns=["x","y"])
query_df = pd.DataFrame(arr_random_query, columns=["x","y"])
df["weapon"] = "a"
df["weapon"]=df.weapon.apply(lambda x:random.choice(weapons))
df.to_parquet("notebook_results/input.parquet")
query_df.to_parquet("notebook_results/query.parquet")

### This seems to work, have to create the algorithm for it

In [ ]:
out = []
for idx, row in query_df.iterrows():

    diffs = (df[["x", "y"]] - row).pipe(lambda df: df**2).sum(axis=1)
    out.append({"dist": math.sqrt(diffs.min()), "weapon": df.loc[diffs.argmin(), "weapon"]})
pd.DataFrame(out).to_csv("notebook_results/out.csv", index=False)

In [46]:
%%timeit
out = []
for idx, row in query_df.iterrows():

    diffs = (df[["x", "y"]] - row).pipe(lambda df: df**2).sum(axis=1)
    out.append({"dist": math.sqrt(diffs.min()), "weapon": df.loc[diffs.argmin(), "weapon"]})
pd.DataFrame(out).to_csv("notebook_results/out.csv", index=False)

2.24 s ± 249 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
# This is 20x better than the original
tree = spatial.KDTree(df[["x","y"]])
out = []
for idx, row in query_df.iterrows():
    out.append({"dist": tree.query(row)[0], "weapon": df.loc[tree.query(row)[1], "weapon"]})
#pd.DataFrame(out).to_csv("notebook_results/out_tree.csv", index=False)


In [14]:
%%timeit
tree = spatial.KDTree(df[["x","y"]])
out = []
for idx, row in query_df.iterrows():
    out.append({"dist": tree.query(row)[0], "weapon": df.loc[tree.query(row)[1], "weapon"]})
#pd.DataFrame(out).to_csv("notebook_results/out_tree.csv", index=False)

379 ms ± 3.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


try with polars

In [8]:
query_df_p = pl.read_parquet("notebook_results/query.parquet")
df_p = pl.read_parquet("notebook_results/input.parquet")

In [9]:
#df_p = df_p.with_row_index("rownum",0)

In [10]:
tree = spatial.KDTree(df_p.select("x","y"))

In [11]:
tree.query(query_df_p[1])[1]

array([766828], dtype=int64)

In [51]:
for row in query_df_p.iter_rows():
    asd = df_p[int(tree.query(row)[1])]["weapon"]

In [32]:
out = []
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0], "weapon": tree.query(row)[1]})

In [17]:
tree = spatial.KDTree(df_p[["x","y"]])
out = []
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0], 
                "weapon": df_p[int(tree.query(row)[1])]["weapon"].item()})

In [16]:
%%timeit
tree = spatial.KDTree(df_p[["x","y"]])
out = []
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0], 
                "weapon": df_p[int(tree.query(row)[1])]["weapon"].item()})

348 ms ± 3.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
